# Week7 Multicollinearity 
## 背景描述  
McDonald和Schwing(1973)提出了一项研究，将总死亡率与气候、社会经济和污染变量联系起来，本研究选择了 15 个自变量列于下表。因变量是以上原因在年龄调整后的总死亡率。我们不对该研究的流行病学方面作评论，而仅仅作为变量选择的说明性例子使用这些数据。  
由此我们构造了 60 个观测的 15 个变量，具体请见下表：

## 数据描述
| 变量名 | 变量含义 | 变量类型 | 变量取值范围 |
| :----------: | :--------: | :----------: | :----------: |
| （自变量1）X1 | 年平均降水量(英寸) | continuous variable | $\mathbb{R}^+$ |
| （自变量2）X2 | 一月平均气温(华氏度) | continuous variable | $\mathbb{R}^+$ |
| （自变量3）X3 | 七月平均气温(华氏度) | continuous variable | $\mathbb{R}^+$ |
| （自变量4）X4 | 占65岁以上人口的比例 | continuous variable | $\mathbb{R}^+$ |
| （自变量5）X5 | 每个家庭人口 | continuous variable | $\mathbb{R}^+$ |
| （自变量6）X6 | 完成的平均学制 | continuous variable | $\mathbb{R}^+$ |
| （自变量7）X7 | 健全的住房单位的百分比 | continuous variable | $\mathbb{R}^+$ |
| （自变量8）X8 | 每平方英里人口 | continuous variable | $\mathbb{R}^+$ |
| （自变量9）X9 | 占非白人人口的比例 | continuous variable | $\mathbb{R}^+$ |
| （自变量10）X10 | 白领工作的就业率 | continuous variable | $\mathbb{R}^+$ |
| （自变量11）X11 | 收入在3000美元以下家庭的百分比 | continuous variable | $\mathbb{R}^+$ |
| （自变量12）X12 | 碳氢化合物的相对污染潜力 | continuous variable | $\mathbb{R}^+$ |
| （自变量13）X13 | 氮氧化物的相对污染潜力 | continuous variable | $\mathbb{R}^+$ |
| （自变量14）X14 | 二氧化硫的相对污染潜力 | continuous variable | $\mathbb{R}^+$ |
| （自变量15）X15 | 相对湿度百分比 | continuous variable | $\mathbb{R}^+$ |
| （因变量）Y | 所有原因经年龄调整的总死亡率 | continuous variable | $\mathbb{R}^+$ |

## 问题
这里使用 $\alpha=0.05$ 的水平
1. 判断变量间是否具有多重共线性.
2. 如果存在多重共线性，如何消除多重共线性/选择变量. 

## 解决方案

**Q1：**  

In [1]:
# Import standard packages
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import math

# Import additional packages
from itertools import combinations
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

alpha = 0.05
p = 15
n = 60

x = pd.read_csv('Air_Pollution.csv')
x.insert(0, 'intercept', np.ones(len(x))) 
data = x.values * 1
df = pd.DataFrame(data)
print(df.head())

X = data[:,0:p+1]
Y = data[:,-1]



    0     1     2     3     4     5     6     7       8     9     10    11  \
0  1.0  36.0  27.0  71.0   8.1  3.34  11.4  81.5  3243.0   8.8  42.6  11.7   
1  1.0  35.0  23.0  72.0  11.1  3.14  11.0  78.8  4281.0   3.5  50.7  14.4   
2  1.0  44.0  29.0  74.0  10.4  3.21   9.8  81.6  4260.0   0.8  39.4  12.4   
3  1.0  47.0  45.0  79.0   6.5  3.41  11.1  77.5  3125.0  27.1  50.2  20.6   
4  1.0  43.0  35.0  77.0   7.6  3.44   9.6  84.6  6441.0  24.4  43.7  14.3   

     12    13     14    15       16  
0  21.0  15.0   59.0  59.0   921.87  
1   8.0  10.0   39.0  57.0   997.88  
2   6.0   6.0   33.0  54.0   962.35  
3  18.0   8.0   24.0  56.0   982.29  
4  43.0  38.0  206.0  55.0  1071.29  


**数据预处理：**

In [2]:
# 对自变量 X 进行标准化
# 自变量 X 的均值
X_mean = []
for i in range(p):
    X_mean.append(np.mean(X[:, i+1])) 

# 自变量 X 的标准差
X_L = []
for i in range(p):
    X_L.append(sum((X[:, i+1] - X_mean[i]) ** 2))  

# 对自变量 X 标准化(截距项不用标准化)
X_std = X * 1.0
X_std[:,1:p+1] = (X[:,1:p+1] - X_mean) / np.sqrt(X_L)



**做多元线性回归分析**

In [3]:
# Do the multiple linear regression
# OLS（endog,exog=None,missing='none',hasconst=None) (endog:因变量，exog=自变量）
model = sm.OLS(Y, X).fit()
Y_hat = model.fittedvalues
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.685
Method:                 Least Squares   F-statistic:                     9.542
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           2.19e-09
Time:                        18:37:55   Log-Likelihood:                -289.03
No. Observations:                  60   AIC:                             610.1
Df Residuals:                      44   BIC:                             643.6
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1763.9979    437.330      4.034      0.000     882.617    2645.379
x1             1.9054      0.924      2.063      0.045       0.044       3.767
x2            -1.9376      1.108     -1.748      0.087      -4.171       0.296
x3            -3.1004      1.902     -1.630      0.110      -6.933       0.732
x4            -9.0652      8.486     -1.068      0.291     -26.168       8.038
x5          -106.8310     69.780     -1.531      0.133    -247.464      33.801
x6           -17.1569     11.860     -1.447      0.155     -41.059       6.746
x7            -0.6511      1.768     -0.368      0.714      -4.214       2.912
x8             0.0036      0.004      0.894      0.376      -0.005       0.012
x9             4.4596      1.327      3.360      0.002       1.785       7.134
x10           -0.1871      1.662     -0.113      0.911      -3.536       3.162
x11           -0.1674      3.227     -0.052      0.959      -6.672       6.337
x12           -0.6722      0.491     -1.369      0.178      -1.662       0.317
x13            1.3401      1.006      1.333      0.190      -0.687       3.367
x14            0.0863      0.148      0.585      0.562      -0.211       0.384
x15            0.1067      1.169      0.091      0.928      -2.250       2.464
==============================================================================
Omnibus:                        2.991   Durbin-Watson:                   2.129
Prob(Omnibus):                  0.224   Jarque-Bera (JB):                2.088
Skew:                           0.359   Prob(JB):                        0.352
Kurtosis:                       3.566   Cond. No.                     4.05e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [4]:
# Do the multiple linear regression
# OLS（endog,exog=None,missing='none',hasconst=None) (endog:因变量，exog=自变量）
model_std = sm.OLS(Y, X_std).fit()
Y_std_hat = model_std.fittedvalues
model_std.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.685
Method:                 Least Squares   F-statistic:                     9.542
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           2.19e-09
Time:                        18:37:55   Log-Likelihood:                -289.03
No. Observations:                  60   AIC:                             610.1
Df Residuals:                      44   BIC:                             643.6
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        940.3585      4.509    208.538      0.000     931.271     949.446
x1           146.1294     70.845      2.063      0.045       3.351     288.908
x2          -151.3453     86.575     -1.748      0.087    -325.826      23.135
x3          -113.4333     69.576     -1.630      0.110    -253.654      26.787
x4          -101.9781     95.465     -1.068      0.291    -294.376      90.419
x5          -110.9860     72.494     -1.531      0.133    -257.088      35.116
x6          -111.3974     77.006     -1.447      0.155    -266.593      43.798
x7           -25.7135     69.812     -0.368      0.714    -166.410     114.983
x8            40.2161     44.979      0.894      0.376     -50.434     130.866
x9           305.5909     90.946      3.360      0.002     122.300     488.881
x10           -6.6313     58.879     -0.113      0.911    -125.295     112.032
x11           -5.3495    103.126     -0.052      0.959    -213.187     202.488
x12         -474.8764    346.905     -1.369      0.178   -1174.016     224.264
x13          476.9329    357.884      1.333      0.190    -244.334    1198.200
x14           42.0025     71.829      0.585      0.562    -102.759     186.764
x15            4.4026     48.236      0.091      0.928     -92.810     101.615
==============================================================================
Omnibus:                        2.991   Durbin-Watson:                   2.129
Prob(Omnibus):                  0.224   Jarque-Bera (JB):                2.088
Skew:                           0.359   Prob(JB):                        0.352
Kurtosis:                       3.566   Cond. No.                         111.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**预判变量间是否存在多重共线性**  

**方法1： 直观判定法**

In [5]:
# 相关系数
r = df.corr()
r

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1.000000,0.092208,0.503273,0.101113,0.263444,-0.490425,-0.490759,-0.003515,0.413204,-0.297291,0.506585,-0.531760,-0.487321,-0.106924,-0.077343,0.509500
2,NaN,0.092208,1.000000,0.346282,-0.398099,-0.209212,0.116284,0.014852,-0.100051,0.453774,0.237992,0.565314,0.350809,0.321014,-0.107810,0.067872,-0.030022
3,NaN,0.503273,0.346282,1.000000,-0.434040,0.262280,-0.238544,-0.415032,-0.060994,0.575309,-0.021412,0.619308,-0.356494,-0.337668,-0.099348,-0.452809,0.277018
4,NaN,0.101113,-0.398099,-0.434040,1.000000,-0.509087,-0.138862,0.065010,0.161991,-0.637821,-0.117715,-0.309771,-0.020486,-0.002082,0.017248,0.112426,-0.174593
5,NaN,0.263444,-0.209212,0.262280,-0.509087,1.000000,-0.395075,-0.410590,-0.184332,0.419410,-0.425723,0.259904,-0.388205,-0.358429,-0.004084,-0.135740,0.357307
6,NaN,-0.490425,0.116284,-0.238544,-0.138862,-0.395075,1.000000,0.552237,-0.243883,-0.208774,0.703196,-0.403338,0.286835,0.224402,-0.234346,0.176491,-0.510988
7,NaN,-0.490759,0.014852,-0.415032,0.065010,-0.410590,0.552237,1.000000,0.181881,-0.410334,0.338745,-0.680680,0.386767,0.348250,0.117952,0.121901,-0.426821
8,NaN,-0.003515,-0.100051,-0.060994,0.161991,-0.184332,-0.243883,0.181881,1.000000,-0.005678,-0.031765,-0.162945,0.120282,0.165312,0.432086,-0.124976,0.265503
9,NaN,0.413204,0.453774,0.575309,-0.637821,0.419410,-0.208774,-0.410334,-0.005678,1.000000,-0.004387,0.704915,-0.025865,0.018385,0.159293,-0.117957,0.643742


1. 当与因变量之间的简单相关系数绝对值很大的自变量在回归方程中没有通过显著性检验时，可初步判断存在严重的多重共线性。

In [6]:
r_xy = np.array(r.iloc[1:p+1][p+1])
print('因变量和每个自变量之间的相关系数: \n', r_xy)

judge_xy = True
for i in range(p):
    if (abs(r_xy[i]) >= 0.5) & (model_std.pvalues[i+1] >= alpha):
        judge_xy = False
        print('自变量 %d 与因变量之间的简单相关系数为: %.4f, tPal: %.4f.' % (i+1, r_xy[i], model_std.pvalues[i+1]))
        
if judge_xy:
    print('\n自变量之间不存在多重共线性。')
else:
    print('\n自变量之间存在多重共线性。')

因变量和每个自变量之间的相关系数: 
 [ 0.50949986 -0.03002188  0.27701762 -0.17459291  0.35730691 -0.51098849
 -0.42682123  0.2655034   0.64374176 -0.28480459  0.41049037 -0.17724211
 -0.07738243  0.42589789 -0.08850055]
自变量 6 与因变量之间的简单相关系数为: -0.5110, tPal: 0.1551.

自变量之间存在多重共线性。


2. 在自变量的相关矩阵中，当自变量间的相关系数较大时会出现多重共线性的问题。

In [7]:
judge_xx = True
for (i, j) in combinations(range(1, p+1), 2):
    if(r.iloc[i][j] >= 0.7):
        judge_xx = False
        print('变量(%d,%d)之间相关系数较大，为：%.4f'% (i, j, r.iloc[i][j]))
        
if judge_xx:
    print('\n自变量之间不存在多重共线性。')
else:
    print('\n自变量之间存在多重共线性。')

变量(6,10)之间相关系数较大，为：0.7032
变量(9,11)之间相关系数较大，为：0.7049
变量(12,13)之间相关系数较大，为：0.9838

自变量之间存在多重共线性。


**方法2：方差扩大因子法**  
1. 计算自变量 $x_j$ 的方差扩大因子 $\mathsf{VIF_j}$，$j=1,\cdots,p$.

In [8]:
# 法1：
c = np.dot(X_std.T, X_std)
C = np.linalg.inv(c)  # 求逆
C_list = []
for i in range(p):
    C_list.append(C[i + 1][i + 1])

# 法2：
vif = [variance_inflation_factor(X_std[:,1:p + 1], i) for i in range(p)]

print('C主对角线元素  方差扩大因子：')
for i in range(p):
    print('%d. %.4f        %.4f' % (i+1, C_list[i], vif[i]))

C主对角线元素  方差扩大因子：
1. 4.1139        4.1139
2. 6.1436        6.1436
3. 3.9678        3.9678
4. 7.4700        7.4700
5. 4.3076        4.3076
6. 4.8605        4.8605
7. 3.9948        3.9948
8. 1.6583        1.6583
9. 6.7796        6.7796
10. 2.8416        2.8416
11. 8.7171        8.7171
12. 98.6399        98.6399
13. 104.9824        104.9824
14. 4.2289        4.2289
15. 1.9071        1.9071


2. 通过 $\mathsf{VIF_j}$ 的大小判断自变量之间是否存在多重共线性.  
如果VIF值大于10说明共线性很严重，这种情况需要处理，如果VIF值在5以下不需要处理，如果VIF介于5~10之间视情况而定。

In [9]:
thres_vif = 5
for i in range(p):
    if vif[i] >= thres_vif:
        print('自变量 x%d 与其余自变量之间存在多重共线性，其中VIF值为：%.4f' % (i + 1, vif[i]))


自变量 x2 与其余自变量之间存在多重共线性，其中VIF值为：6.1436
自变量 x4 与其余自变量之间存在多重共线性，其中VIF值为：7.4700
自变量 x9 与其余自变量之间存在多重共线性，其中VIF值为：6.7796
自变量 x11 与其余自变量之间存在多重共线性，其中VIF值为：8.7171
自变量 x12 与其余自变量之间存在多重共线性，其中VIF值为：98.6399
自变量 x13 与其余自变量之间存在多重共线性，其中VIF值为：104.9824


**方法3：特征值判定法**  
1. 计算自变量 $x_j$ 的条件数 $\kappa_j = \sqrt{\frac{\lambda_1}{\lambda_j}}$，$j=1,\cdots,p$.

In [10]:
corr = np.corrcoef(X_std[:,1:p+1], rowvar = 0) # 相关系数矩阵
w, v = np.linalg.eig(corr) # 特征值 & 特征向量

kappa = []
for i in range(p):
    kappa.append(np.sqrt(max(w) / w[i]))
    print('特征值%d: %.4f, kappa%d: %.4f' %(i + 1, w[i], i + 1, kappa[i]))

特征值1: 4.5284, kappa1: 1.0000
特征值2: 2.7548, kappa2: 1.2821
特征值3: 2.0545, kappa3: 1.4846
特征值4: 1.3484, kappa4: 1.8326
特征值5: 1.2232, kappa5: 1.9241
特征值6: 0.9604, kappa6: 2.1714
特征值7: 0.6127, kappa7: 2.7185
特征值8: 0.4720, kappa8: 3.0974
特征值9: 0.3709, kappa9: 3.4944
特征值10: 0.0049, kappa10: 30.5051
特征值11: 0.0460, kappa11: 9.9176
特征值12: 0.2164, kappa12: 4.5746
特征值13: 0.1664, kappa13: 5.2175
特征值14: 0.1140, kappa14: 6.3030
特征值15: 0.1270, kappa15: 5.9712


2. 通过 $\kappa_p$ 的大小判断自变量之间是否存在多重共线性以及多重共线性的严重程度.  
记 $\kappa=\lambda_{max}/ \lambda_{min}$，从实际应用的角度,一般若 $\kappa<100$，则认为多重共线性的程度很小，若是 $100<=\kappa<=1000$，则认为存在一般程度上的多重共线性，若是 $\kappa>1000$，则就认为存在严重的多重共线性.  
$\kappa >= c_{\kappa}$时，自变量之间存在多重共线性，$c_{\kappa}$常见取值为10，100，1000.

In [11]:
thres_kappa = 10
if np.max(kappa) >= thres_kappa:
    print('设计矩阵 X 存在多重共线性，其中kappa值为：%.4f' % np.max(kappa))
else:
    print('设计矩阵 X 不存在多重共线性，其中kappa值为：%.4f' % np.max(kappa))

设计矩阵 X 存在多重共线性，其中kappa值为：30.5051


**Q2：**  
消除多重共线性：
1. 剔除不重要的解释变量
2. 增大样本量

**方法1：剔除不重要的解释变量**

In [12]:
# 利用VIF删除导致高共线性的变量
col = list(range(X_std.shape[1]))
dropped = True
while dropped:
    dropped = False
    vif_drop = [variance_inflation_factor(X_std[:,col], i) for i in range(X_std[:,col].shape[1])]
    maxvif = max(vif_drop)
    maxix = vif_drop.index(maxvif)
    if maxvif > thres_vif:
        del col[maxix]
        dropped = True
    
    if dropped:
        print('剔除剩余变量中第 %d 列变量：' % maxix, '剩余变量：', col)
        
        # 利用 AIC、BIC 准则做变量选择的一个参考
        X_std_vif = X_std[:, col]
        model_vif = sm.OLS(Y, X_std_vif).fit()
        print('此时模型的AIC值为：%.4f'% model_vif.aic)
 

剔除剩余变量中第 13 列变量： 剩余变量： [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15]
此时模型的AIC值为：610.4349
剔除剩余变量中第 11 列变量： 剩余变量： [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15]
此时模型的AIC值为：608.4349
剔除剩余变量中第 4 列变量： 剩余变量： [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 12, 14, 15]
此时模型的AIC值为：607.3130


In [13]:
# Do the multiple linear regression
X_std_vif = X_std[:, col]
model_vif = sm.OLS(Y, X_std_vif).fit()
model_vif.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.752
Model:                            OLS   Adj. R-squared:                  0.688
Method:                 Least Squares   F-statistic:                     11.86
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           1.67e-10
Time:                        18:37:55   Log-Likelihood:                -290.66
No. Observations:                  60   AIC:                             607.3
Df Residuals:                      47   BIC:                             634.5
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        940.3585      4.483    209.767      0.000     931.340     949.377
x1           114.4016     53.971      2.120      0.039       5.827     222.977
x2          -121.1191     57.930     -2.091      0.042    -237.660      -4.579
x3           -96.5763     65.378     -1.477      0.146    -228.101      34.948
x4           -61.0044     54.900     -1.111      0.272    -171.449      49.440
x5           -63.9241     70.253     -0.910      0.368    -205.256      77.407
x6           -30.6498     53.258     -0.575      0.568    -137.792      76.492
x7            43.5835     44.288      0.984      0.330     -45.512     132.679
x8           351.4467     66.359      5.296      0.000     217.950     484.944
x9           -28.6629     55.417     -0.517      0.607    -140.147      82.822
x10          -31.6548     61.719     -0.513      0.610    -155.818      92.508
x11          113.3616     46.157      2.456      0.018      20.506     206.217
x12           -2.5819     46.842     -0.055      0.956     -96.817      91.653
==============================================================================
Omnibus:                        6.868   Durbin-Watson:                   2.055
Prob(Omnibus):                  0.032   Jarque-Bera (JB):                6.115
Skew:                           0.624   Prob(JB):                       0.0470
Kurtosis:                       3.942   Cond. No.                         21.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
# 后退法
col0 = list(range(X_std.shape[1]))
col1 = col0 * 1
dropped1 = True
aic_model = sm.OLS(Y, X_std).fit().aic
while dropped1:
    X_std_aic = X_std[:, col1]
    model_aic = sm.OLS(Y, X_std_aic).fit().aic
    aic = []
    for i in range(len(col1)):  # 改动：i的范围
        col2 = col1 * 1  # 改动：col1赋值给col2
        del col2[i]
        aic.append(sm.OLS(Y, X_std[:, col2]).fit().aic)
    minaic = min(aic[1:len(aic)])
    minaic_rank = aic.index(minaic)    # 被剔除的变量下标
    minaic_ix = col1[minaic_rank]      # 被剔除的变量编号
    if minaic < model_aic:
        del col1[minaic_rank]
    else:
        dropped1 = False
    
    if dropped1:
        print('剔除剩余变量中第 %d 列变量：' % minaic_ix, '剩余变量：', col1)
        print('此时模型的AIC值为：%.4f'% minaic)

        

剔除剩余变量中第 11 列变量： 剩余变量： [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15]
此时模型的AIC值为：608.0644
剔除剩余变量中第 15 列变量： 剩余变量： [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14]
此时模型的AIC值为：606.0785
剔除剩余变量中第 10 列变量： 剩余变量： [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14]
此时模型的AIC值为：604.0972
剔除剩余变量中第 7 列变量： 剩余变量： [0, 1, 2, 3, 4, 5, 6, 8, 9, 12, 13, 14]
此时模型的AIC值为：602.3304
剔除剩余变量中第 14 列变量： 剩余变量： [0, 1, 2, 3, 4, 5, 6, 8, 9, 12, 13]
此时模型的AIC值为：600.6631
剔除剩余变量中第 8 列变量： 剩余变量： [0, 1, 2, 3, 4, 5, 6, 9, 12, 13]
此时模型的AIC值为：599.9077


In [15]:
# Do the multiple linear regression
X_std_aic = X_std[:, col1]
model_aic = sm.OLS(Y, X_std_aic).fit()
model_aic.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.758
Model:                            OLS   Adj. R-squared:                  0.714
Method:                 Least Squares   F-statistic:                     17.36
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           1.48e-12
Time:                        18:37:56   Log-Likelihood:                -289.95
No. Observations:                  60   AIC:                             599.9
Df Residuals:                      50   BIC:                             620.9
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        940.3585      4.296    218.907      0.000     931.730     948.987
x1           142.3767     64.217      2.217      0.031      13.394     271.360
x2          -176.6846     54.338     -3.252      0.002    -285.826     -67.544
x3          -121.4650     51.116     -2.376      0.021    -224.134     -18.796
x4          -122.8465     80.319     -1.529      0.132    -284.172      38.479
x5          -142.7315     62.100     -2.298      0.026    -267.463     -18.000
x6          -152.0741     45.852     -3.317      0.002    -244.172     -59.977
x7           319.4820     66.397      4.812      0.000     186.120     452.844
x8          -651.4948    225.538     -2.889      0.006   -1104.501    -198.489
x9           665.9154    216.924      3.070      0.003     230.212    1101.619
==============================================================================
Omnibus:                        0.997   Durbin-Watson:                   2.184
Prob(Omnibus):                  0.607   Jarque-Bera (JB):                0.565
Skew:                           0.229   Prob(JB):                        0.754
Kurtosis:                       3.130   Cond. No.                         72.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
# 方差扩大因子法
print('方差扩大因子：')
for i in range(X_std_vif.shape[1] - 1):
    print('%.4f' % vif_drop[i + 1])

print('\n')
for i in range(X_std_vif.shape[1] - 1):
    if vif_drop[i] >= thres_vif:
        print('自变量 x%d 与其余自变量之间存在多重共线性，其中VIF值为: %.4f' % (i + 1, vif_drop[i + 1]))

方差扩大因子：
2.4158
2.7832
3.5449
2.4997
4.0933
2.3524
1.6267
3.6520
2.5470
3.1592
1.7669
1.8198




In [17]:
# 特征值判定法
corr_ = np.corrcoef(X_std_vif[:,1:p+1], rowvar = 0) # 相关系数矩阵
w_, v_ = np.linalg.eig(corr_) # 特征值 & 特征向量

kappa_ = []
for i in range(X_std_vif.shape[1] - 1):
    kappa_.append(np.sqrt(w_[0] / w_[i]))
    print('特征值%d: %.4f, kappa%d: %.4f' %(i + 1, w_[i], i + 1, kappa_[i]))
    
if max(kappa_) >= thres_kappa:
    print('\n设计矩阵 X 存在多重共线性，其中 kappa 值为: %.4f' % max(kappa_))
else:
    print('\n设计矩阵 X 不存在多重共线性，其中 kappa 值为: %.4f' % max(kappa_))

特征值1: 3.6174, kappa1: 1.0000
特征值2: 1.9819, kappa2: 1.3510
特征值3: 1.7561, kappa3: 1.4353
特征值4: 1.1025, kappa4: 1.8114
特征值5: 0.9685, kappa5: 1.9326
特征值6: 0.8119, kappa6: 2.1108
特征值7: 0.5317, kappa7: 2.6083
特征值8: 0.4441, kappa8: 2.8540
特征值9: 0.3198, kappa9: 3.3634
特征值10: 0.2123, kappa10: 4.1277
特征值11: 0.1246, kappa11: 5.3887
特征值12: 0.1292, kappa12: 5.2923

设计矩阵 X 不存在多重共线性，其中 kappa 值为: 5.3887


综上，剔除原始数据的第 4、11 和 13 个变量，可以一定程度上消除变量间的多重共线性。

**方法2：增大样本量**  
这里不用增大样本量，原因有二：  
其一，这个数据集中数据量是充足的，而且不是因为样本量过少而导致的多重共线性问题，更多是因为这个变量之间的相关性很强造成的；  
其二，增加变量的方法，只是在于采集数据时，如果样本量过小可能会产生多重共线性的问题，因此需要采集足够多的样本。在实际分析阶段，往往无法增加样本量。

## 第七周练习题
数据集：Project7.csv(内附文档)  
统计方法：Multicollinearity  
软件：Jupyter Notebook  
作业发到钉钉群  
Deadline：下周一晚上10：00之前交  
注：要有完整的解题过程，不能只有代码